# DS Experiments to verify theory for L-inf norm - FMNIST

In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division
from __future__ import print_function

import sys, os, gc, math
import numpy as np
from scipy.fftpack import dct,idct
from keras.datasets import fashion_mnist
from PIL import Image
from multiprocessing import Pool
import time



sys.path.append('../')

from models.util import *


#Seed used for choosing classes, training points, and test points.
#SEED = 14
SEED=11

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
num_samples = 500
sqrt_n = 28
input_shape=(sqrt_n,sqrt_n)
n = sqrt_n*sqrt_n
k = 40
c=2.0

In [3]:
#Load Cifar10 data
(X_train, _), (X_test, _) = fashion_mnist.load_data()
m_data = np.concatenate((X_train,X_test))

In [4]:
#Normalize the data
m_data = m_data/255.0

In [5]:
#Check FMNIST results for 1000 random images - BP
subset_idx = np.random.choice(np.arange(m_data.shape[0]),num_samples)
m_data_sub_bp = m_data[subset_idx]
m_data_y_bp = np.zeros((num_samples,sqrt_n,sqrt_n))
for i in range(num_samples):
    #first sample an element from the data
    x = m_data_sub_bp[i,:,:].flatten()
    x_hat = dct(x,norm='ortho')
    x_k = idct(get_topk_vec(x_hat,k=k))
    e = np.random.uniform(size=n)
    y = x_k + e
    m_data_y_bp[i,:,:] = y.reshape((sqrt_n,sqrt_n))  
    m_data_sub_bp[i,:,:] = x_k.reshape((sqrt_n,sqrt_n)) 

In [6]:
#Form the matrix F
n = 28*28
F = get_matrix(n,tf='dct')

In [7]:
eta_1 = 1.0
eta_2 = np.linalg.norm(dct(np.ones(n), norm='ortho'), ord=np.inf)

In [8]:
def approximate(z):
    x_hat = dantzig(z,F.T,n=n,eta_1=eta_1,eta_2=eta_2 )
    return x_hat

In [ ]:
%%capture three
errors_l2_bp = np.zeros(m_data_y_bp.shape[0])
errors_l1_bp = np.zeros(m_data_y_bp.shape[0])
eta_bp = np.zeros((m_data_y_bp.shape[0]))
diff_l2_bp = np.zeros(m_data_y_bp.shape[0])
diff_l1_bp = np.zeros(m_data_y_bp.shape[0])

for i in range(num_samples):
    y = m_data_y_bp[i,:,:].flatten()
    x = m_data_sub_bp[i,:,:].flatten()
    
    #Get actual top k and bottom k
    x_hat =  dct(x, norm='ortho')  

    x_hat_approx = dantzig(y,F.T,n=n,eta_1=eta_1,eta_2=eta_2 )



    #Note the errors
    errors_l2_bp[i] = np.linalg.norm(x_hat.flatten()- x_hat_approx.flatten()) 
    errors_l1_bp[i] = np.linalg.norm(x_hat.flatten()- x_hat_approx.flatten(),ord=1) 


    #Get the multiplicative constant
    c_l2 = 6*np.sqrt(k)*eta_2
    c_l1 = 4*k*eta_2
    
    
    #Calculate the difference from the upper bound
    diff_l2_bp[i] = c_l2 - errors_l2_bp[i] 
    diff_l1_bp[i] = c_l1 - errors_l1_bp[i]

In [14]:
print(np.mean(errors_l2_bp), 
      np.mean(diff_l2_bp), 
      np.mean(errors_l1_bp), 
      np.mean(diff_l1_bp))

274.21534916293433 788.3099446536411 1824.3097926449734 2655.690207355027


In [11]:
e_norms = np.zeros(m_data_y_bp.shape)
for i in range(num_samples):
    y = m_data_y_bp[i,:,:].flatten()
    x = m_data_sub_bp[i,:,:].flatten()
    e_norms[i] = np.linalg.norm(y - x,ord=np.inf)

In [12]:
np.mean(e_norms)

0.9987032963044421

In [ ]:
mnist_tup_bp = (m_data_y_bp, m_data_sub_bp, errors_l2_bp, errors_l1_bp, diff_l2_bp, diff_l1_bp )

import pickle
with open('data/cifar_theory_socp_l2.pickle', 'wb') as f:
    pickle.dump(mnist_tup_bp, f)
